In [ ]:
# 导入必要的库和模块
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

import time
import tarfile

# 模拟耗时操作（如数据下载）
for i in range(100):
    time.sleep(0.1)  # 模拟耗时操作，每次休眠0.1秒
    print(f"当前进度: {i+1}/100")  # 输出当前进度，显示下载进度

# 解压 CIFAR-10 数据集
with tarfile.open('cifar-10-python.tar.gz', 'r:gz') as tar:
    tar.extractall('./data')  # 将压缩文件解压到./data目录

print("解压完成")  # 提示解压完成

# 定义数据预处理转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为 PyTorch 张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 对图像进行标准化
])

# 加载 CIFAR-10 训练集
trainset = torchvision.datasets.CIFAR10(
    root='./data',  # 指定数据集根目录
    train=True,     # 表示加载训练集
    download=False, # 数据集已经存在，不需要下载
    transform=transform # 应用预处理转换
)
trainloader = torch.utils.data.DataLoader(
    trainset,       # 加载的训练集
    batch_size=4,   # 每次加载4个样本
    shuffle=True,   # 在每个训练周期开始时打乱数据顺序
    num_workers=2   # 使用两个子进程来加载数据
)

# 加载 CIFAR-10 测试集
testset = torchvision.datasets.CIFAR10(
    root='./data',  # 指定数据集根目录
    train=False,    # 表示加载测试集
    download=False, # 数据集已经存在，不需要下载
    transform=transform # 应用预处理转换
)
testloader = torch.utils.data.DataLoader(
    testset,        # 加载的测试集
    batch_size=4,   # 每次加载4个样本
    shuffle=False,  # 不打乱测试数据顺序
    num_workers=2   # 使用两个子进程来加载数据
)

# 定义数据集的类别名称
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 定义 CNN 模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 第一个卷积层，输入通道3，输出通道6，卷积核大小5
        self.pool = nn.MaxPool2d(2, 2)   # 最大池化层，窗口大小2，步长2
        self.conv2 = nn.Conv2d(6, 16, 5) # 第二个卷积层，输入通道6，输出通道16，卷积核大小5
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 第一个全连接层，输入特征数16*5*5，输出120
        self.fc2 = nn.Linear(120, 84)     # 第二个全连接层，输入120，输出84
        self.fc3 = nn.Linear(84, 10)      # 第三个全连接层，输入84，输出10（类别数）

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 应用第一个卷积层，激活函数ReLU，然后池化
        x = self.pool(F.relu(self.conv2(x)))  # 应用第二个卷积层，激活函数ReLU，然后池化
        x = x.view(-1, 16 * 5 * 5)           # 将张量展平为一维向量
        x = F.relu(self.fc1(x))              # 应用第一个全连接层和ReLU激活
        x = F.relu(self.fc2(x))              # 应用第二个全连接层和ReLU激活
        x = self.fc3(x)                      # 应用第三个全连接层（输出层）
        return x

# 检查是否有可用的 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # 输出当前使用的设备（CPU或GPU）

net = Net().to(device)  # 创建模型实例并移动到 GPU 上

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)  # 使用随机梯度下降优化器

# 计时开始
start_time = time.time()

# 记录每个周期的准确率和损失
train_accuracies = []
train_losses = []
test_accuracies = []

# 训练模型
for epoch in range(30):  # 训练30个周期
    running_loss = 0.0  # 初始化运行损失
    correct = 0         # 初始化正确预测计数
    total = 0           # 初始化总样本计数
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data  # 获取输入数据和标签
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到 GPU 上
        optimizer.zero_grad()  # 清除梯度
        outputs = net(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        
        running_loss += loss.item()  # 累加损失
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        total += labels.size(0)  # 累加总样本数
        correct += (predicted == labels).sum().item()  # 累加正确预测数
    
    # 记录训练准确率和损失
    train_loss = running_loss / (i + 1)
    train_accuracy = 100 * correct / total
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    # 输出每个周期的损失和准确率
    print(f'Epoch {epoch + 1}, Loss: {train_loss}, Accuracy: {train_accuracy}%')

print('Finished Training')  # 提示训练完成

# 计时结束
end_time = time.time()

# 测试模型
correct = 0  # 初始化正确预测计数
total = 0    # 初始化总样本计数
class_correct = list(0. for i in range(10))  # 每个类别的正确预测计数
class_total = list(0. for i in range(10))    # 每个类别的总样本计数

with torch.no_grad():  # 禁用梯度计算
    for data in testloader:
        images, labels = data  # 获取测试数据和标签
        images, labels = images.to(device), labels.to(device)  # 将数据移动到 GPU 上
        outputs = net(images)  # 前向传播
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        total += labels.size(0)  # 累加总样本数
        correct += (predicted == labels).sum().item()  # 累加正确预测数
        
        # 统计每个类别的正确预测和总样本数
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# 记录测试准确率
test_accuracy = 100 * correct / total
test_accuracies.append(test_accuracy)

# 输出测试准确率
print(f'Test Accuracy: {test_accuracy}%')

# 输出每个类别的准确率
for i in range(10):
    if class_total[i] != 0:
        print(f'Accuracy of {classes[i]} : {100 * class_correct[i] / class_total[i]}%')
    else:
        print(f'Accuracy of {classes[i]}: 0% (No samples in test set)')

# 输出训练时间
print(f'Training Time: {(end_time - start_time):.2f} seconds')

# 绘制训练和测试准确率图表
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Test Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.tight_layout()
plt.show()